In [10]:
import pandas as pd
from scipy.io import savemat
import matlab.engine

def create_segment(shift):
    n_initial_240s = 22 + shift
    n_150s = 19
    n_final_240s = 1344 - n_initial_240s - n_150s
    
    segment = [240] * n_initial_240s + [150] * n_150s + [240] * n_final_240s
    return segment

def generate_sequence(length):
    seq = []
    value = 0
    for _ in range(length):
        seq.append(value)
        value += 1/96
        if round(value, 2) >= 14:
            value = 0
    return seq

def save_to_mat(data):
    df = pd.DataFrame(data, columns=['Value'])
    df['Sequence'] = generate_sequence(len(df))

    # Validate the data shape
    if df.shape != (1344, 2):
        raise ValueError(f"Unexpected data shape: {df.shape}. Expected (1344, 2).")
    
    # Create combined 2D list
    combined_data = df[['Sequence', 'Value']].values.tolist()
    
    # Save this combined list as a single variable in the .mat file
    data_dict_tank3 = {'KLa3_Setpoints_14Day': combined_data}
    data_dict_tank4 = {'KLa4_Setpoints_14Day': combined_data}
    data_dict_tank5 = {'KLa5_Setpoints_14Day': combined_data}
    
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa5_Setpoints_14Day.mat', data_dict_tank5)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa4_Setpoints_14Day.mat', data_dict_tank4)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa3_Setpoints_14Day.mat', data_dict_tank3)

def run_matlab_model(iteration):
    eng = matlab.engine.start_matlab()

    # Save the 'iteration' value as a MATLAB datafile
    iteration_data = {"iteration": iteration}
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/iteration.mat', iteration_data)

    eng.cd('/Users/aya/github/WWDR/ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1', nargout=0)
    eng.run('run_TSaeration_DRbsm1.m', nargout=0)
    eng.quit()

# Get the number of iterations
n_segments = int(input("Enter the number of iterations: "))

shift = 0
for i in range(n_segments):
    data = create_segment(shift)
    save_to_mat(data)
    run_matlab_model(i + 1)  # Pass the current iteration count, starting from 1
    shift += 4


 
Running BSM1 to steady state! Solver = ode15s and Simulink model = benchmarkss
**************************************************************************
 
Steady state achieved. Initializing all state variables to steady state values.
 
Simulating BSM1 with dynamic influent (i) in closed loop (Tempmodel = 1)! Solver = ode45 and Simulink model = benchmark
*****************************************************************************************************************
 
Start time for simulation (hour:min:sec) = 12  28  44
Simulating BSM1 with dynamic influent (ii) in closed loop (Tempmodel = 1)! Solver = ode45 and Simulink model = benchmark
*****************************************************************************************************************
 
Start time for simulation (hour:min:sec) = 12  29  10
Dynamic closed loop BSM1 simulation finished!
End time for simulation (hour:min:sec) = 12  29  37
 
 
Running BSM1 to steady state! Solver = ode15s and Simulink model = benchmarks

In [14]:
import pandas as pd
from scipy.io import savemat
import matlab.engine

def create_segment(shift):
    n_initial_240s = 22 + shift  
    n_150s = 19 
    n_final_240s = 96 - n_initial_240s - n_150s  # This will ensure the segment has 96 lines
    
    # Create one segment of 96 lines
    one_segment = [240] * n_initial_240s + [150] * n_150s + [240] * n_final_240s
    
    # Stack 14 copies of the segment to make it 1344 lines
    final_segment = one_segment * 14
    
    return final_segment


def generate_sequence(length):
    seq = []
    value = 0
    for _ in range(length):
        seq.append(value)
        value += 1/96
        if round(value, 2) >= 14:
            value = 0
    return seq

def save_to_mat(data):
    df = pd.DataFrame(data, columns=['Value'])
    df['Sequence'] = generate_sequence(len(df))

    # Validate the data shape
    if df.shape != (1344, 2):
        raise ValueError(f"Unexpected data shape: {df.shape}. Expected (1344, 2).")
    
    # Create combined 2D list
    combined_data = df[['Sequence', 'Value']].values.tolist()

    # Convert back to DataFrame for easy indexing
    combined_df = pd.DataFrame(combined_data, columns=['Sequence', 'Value'])

    # Find indices where Value is 150
    indices = combined_df.index[combined_df['Value'] == 150].tolist()
    print(f"Indices where Value is 150: {indices}")

    # Save this combined list as a single variable in the .mat file
    data_dict_tank3 = {'KLa3_Setpoints_14Day': combined_data}
    data_dict_tank4 = {'KLa4_Setpoints_14Day': combined_data}
    data_dict_tank5 = {'KLa5_Setpoints_14Day': combined_data}
    
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa5_Setpoints_14Day.mat', data_dict_tank5)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa4_Setpoints_14Day.mat', data_dict_tank4)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa3_Setpoints_14Day.mat', data_dict_tank3)

def run_matlab_model(iteration):
    eng = matlab.engine.start_matlab()

    # Save the 'iteration' value as a MATLAB datafile
    iteration_data = {"iteration": iteration}
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/iteration.mat', iteration_data)

    eng.cd('/Users/aya/github/WWDR/ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1', nargout=0)
    eng.run('run_TSaeration_DRbsm1.m', nargout=0)
    eng.quit()

# Get the number of iterations
n_segments = int(input("Enter the number of iterations: "))

shift = 0
for i in range(n_segments):
    data = create_segment(shift)
    save_to_mat(data)
    run_matlab_model(i + 1)  # Pass the current iteration count, starting from 1
    shift += 4


Indices where Value is 150: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 982, 983, 984, 985, 986, 987, 988, 989, 